In [1]:
import sys
sys.path.insert(0, "..")

In [2]:
from utils import access_raster
from glob import glob
import re
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from copy import deepcopy
from scipy.stats import gaussian_kde
import seaborn as sns
import random
from scipy import optimize
from osgeo import gdal

import rasterio
from rasterio.transform import from_origin
from rasterio.warp import transform_geom
from shapely.geometry import shape

In [3]:
basepath = "/home/abetatos/Escritorio/TFM_Dataset/3.Basedataset/"
hs_path = basepath + "HS"

is_tiff = lambda x: re.search(".tiff?$", x)

def get_dataframe(path, key): 
    files = filter(is_tiff, glob(path + "/**/*", recursive=True))
    df = pd.DataFrame(files, columns=[key])
    df['file'] = df[key].apply(lambda p: Path(p).parts[-2])
    df = df.set_index("file")
    return df

In [4]:

df = get_dataframe(hs_path, "HS")
len(df)

16

In [12]:
def get_area(raster_path):
    array = access_raster(raster_path)['array']
    points = array[array!=-9999.]
    return len(points)*25/1e6

# def get_area(raster_path):
#     with rasterio.open(raster_path) as src:
#         # Get the CRS (coordinate reference system) of the raster
#         crs = src.crs

#         # Get the transform (affine matrix) of the raster
#         transform = src.transform

#         # Read the raster as a numpy array
#         raster_data = src.read(1)  # Ass
        
#         pixel_width = abs(transform.a)
#         pixel_height = abs(transform.e)
#         pixel_area_m2 = pixel_width * pixel_height

#         # Convert square meters to square kilometers
#         pixel_area_km2 = pixel_area_m2 / 1e6

#         valid_pixels = (raster_data != src.nodata).sum()

#         return valid_pixels * pixel_area_km2

In [13]:
areas = []
for index, series in df.iterrows(): 
    print(round(get_area(series['HS']), 2), index)

    areas.append([round(get_area(series['HS']), 2), index])

109.54 snowmap_20160309_ads_100_extent_maskiert_nosnow0_noausreisser
4.48 HS_Saflischpass_20220512_1m_epsg2056
2.26 HS_Laucherenalp_20220511_1m_epsg2056
108.55 snowmap_2014_ads_100_extent_maskiert_nosnow0_noausreisser
161.08 HS_Davos_20200406_2m_epsg2056
148.78 HS_Davos_20211604_2m_epsg2056
125.71 snowmap_2013_ads_80_extent_maskiert_nosnow0_noausreisser
126.48 snowmap_2012_ads_80_extent_maskiert_nosnow0_noausreisser
109.31 snowmap_2015_ads_100_extent_maskiert_nosnow0_noausreisser
110.8 snowmap_20160126_ads_100_extent_maskiert_nosnow0_noausreisser
6.55 HS_c_20220223_1m_epsg2056
369.19 HS_Davos_20170316_2m_epsg2056
34.47 HS_Davos_20180411_2m_epsg2056
109.96 snowmap_20160420_ads_100_extent_maskiert_nosnow0_noausreisser
1.14 HS_Laucherenalp_20220224_2m_epsg2056
122.51 snowmap_2010_ads_80_extent_maskiert_nosnow0_noausreisser


In [11]:
pd.DataFrame(areas).to_csv("areas.csv")


In [13]:
path = "/home/abetatos/Escritorio/TFM_Dataset/Dataset_20230523/256pixel_4ro/Data"
import os
from tqdm import tqdm
import pickle

lengths = 0
lengthsx = 0

for filename in tqdm(os.listdir(path)):
    if "HS_Davos_20170316_2m_epsg2056" in filename: 
        continue

    filepath = os.path.join(path, filename)
    with open(filepath, 'rb') as f:
        array = pickle.load(f)

    x = array[-1][array[-1]>=0]
    lengths+=len(x)
    lengthsx+=len(x[x<0.2])

100%|██████████| 4050/4050 [00:00<00:00, 4374.79it/s]


In [14]:
#WITHOUT 1
lengthsx, lengths, lengthsx/lengths*100

(11897592, 178928626, 6.649350786385629)